In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow.keras as keras

from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.models import load_model #save and load models
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import average_precision_score
import tensorflow.keras.backend as k

2025-01-22 09:36:37.847607: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-22 09:36:37.848079: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-22 09:36:37.850194: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-22 09:36:37.855948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737549397.865666   83044 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737549397.86

In [2]:
df = pd.read_csv('data/adult.data',  names=[
        "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
        "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
        "Hours per week", "Country", "Target"],
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")

In [3]:
df1 = pd.read_csv("data/adult.test",
    names=[
        "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
        "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
        "Hours per week", "Country", "Target"],
        sep=r'\s*,\s*',
        engine='python',\
        na_values="?")

In [4]:
data = pd.concat([df, df1], ignore_index= True)

In [5]:
names_cloud = data.columns.tolist()
print(names_cloud)
X = np.array(data[names_cloud])
print(X.shape)

['Age', 'Workclass', 'fnlwgt', 'Education', 'Education-Num', 'Martial Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Capital Gain', 'Capital Loss', 'Hours per week', 'Country', 'Target']
(48842, 15)


In [6]:
y = (data['Target'].map({"<=50K":0,">50K":1})).values
print(pd.value_counts(pd.Series(y)))
data.drop('Target',axis=1, inplace =True,)

0    37155
1    11687
Name: count, dtype: int64


/tmp/ipykernel_83044/3481380141.py:2: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  print(pd.value_counts(pd.Series(y)))


In [7]:
categorical_features = data.select_dtypes(include=['object']).columns
print(categorical_features)
ohc_category = ['Workclass', 'Education', 'Martial Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Country']
df_ohc = pd.get_dummies(data, columns = ohc_category)
print(df_ohc.shape)
df_ohc.head()

Index(['Workclass', 'Education', 'Martial Status', 'Occupation',
       'Relationship', 'Race', 'Sex', 'Country'],
      dtype='object')
(48842, 105)


,Age,fnlwgt,Education-Num,Capital Gain,Capital Loss,Hours per week,Workclass_Federal-gov,Workclass_Local-gov,Workclass_Never-worked,Workclass_Private,...,Country_Portugal,Country_Puerto-Rico,Country_Scotland,Country_South,Country_Taiwan,Country_Thailand,Country_Trinadad&Tobago,Country_United-States,Country_Vietnam,Country_Yugoslavia
0,39,77516,13,2174,0,40,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,50,83311,13,0,0,13,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,38,215646,9,0,0,40,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
3,53,234721,7,0,0,40,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
4,28,338409,13,0,0,40,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [8]:
names_x = df_ohc.columns.tolist()
print("Target Variable: Target")
print("Predictors: "+str(names_x))
x = np.array(df_ohc[names_x])
print("Number of data samples : {0:d}".format(x.shape[0]))
print("Number of Predictor Features : {0:d}".format(x.shape[1]))

Target Variable: Target
Predictors: ['Age', 'fnlwgt', 'Education-Num', 'Capital Gain', 'Capital Loss', 'Hours per week', 'Workclass_Federal-gov', 'Workclass_Local-gov', 'Workclass_Never-worked', 'Workclass_Private', 'Workclass_Self-emp-inc', 'Workclass_Self-emp-not-inc', 'Workclass_State-gov', 'Workclass_Without-pay', 'Education_10th', 'Education_11th', 'Education_12th', 'Education_1st-4th', 'Education_5th-6th', 'Education_7th-8th', 'Education_9th', 'Education_Assoc-acdm', 'Education_Assoc-voc', 'Education_Bachelors', 'Education_Doctorate', 'Education_HS-grad', 'Education_Masters', 'Education_Preschool', 'Education_Prof-school', 'Education_Some-college', 'Martial Status_Divorced', 'Martial Status_Married-AF-spouse', 'Martial Status_Married-civ-spouse', 'Martial Status_Married-spouse-absent', 'Martial Status_Never-married', 'Martial Status_Separated', 'Martial Status_Widowed', 'Occupation_Adm-clerical', 'Occupation_Armed-Forces', 'Occupation_Craft-repair', 'Occupation_Exec-managerial', 

## Modelo Alvo original

In [11]:
x = x.astype('float32')
scaler = MinMaxScaler()

# x = x/255
x = scaler.fit_transform(x)
batch_size = 32 #upto us
epochs = 100
lrate = 0.001
decay = 1e-7
data_size = 10000
ns = 5 #number of shadow models for one data_size
nh = 5 #number of hidden layers
nout = 1
seed = 7
np.random.seed(seed)
sh = np.arange(x.shape[0])
np.random.shuffle(sh)
target_rep = np.zeros((1,x.shape[0]))
target_rep[0,:] = sh
print(sh)

[23509 38928 23796 ...   919 38467 10742]


In [12]:
k.clear_session()

xtr_target = x[sh[:data_size]]
ytr_target = y[sh[:data_size]]
xts_target = x[sh[data_size:data_size*2]]
yts_target = y[sh[data_size:2*data_size]]

shadow_rep = np.zeros((20,x.shape[0]-2*data_size))
sh1 = sh[2*data_size:]
xtr_att = np.zeros((2*data_size*ns,1))
ytr_att = np.zeros((2*data_size*ns,1))
xtr_att_truelabels = np.zeros((2*data_size*ns,))

model_target = Sequential()
model_target.add(Dense(nh, input_shape =(x.shape[1],), activation='sigmoid', name = 'hidden'))
model_target.add(Dense(1, activation='sigmoid', name = 'output'))
opt = keras.optimizers.Adam(learning_rate=lrate, decay=decay)
model_target.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
print(model_target.summary())
hist_target = model_target.fit(xtr_target, ytr_target,
                  batch_size = batch_size,
                  epochs = epochs,
                  validation_data=(xts_target, yts_target), shuffle=True, verbose=0)
print('\n\nFor target model with training datasize = %d'%data_size)
print('Training accuracy = %f'%hist_target.history['accuracy'][-1])
print('Validation accuracy = %f'%hist_target.history['val_accuracy'][-1])
model_target_name = 'UCI_Adult_target_'+str(data_size)+'.h5'

model_target.save(model_target_name)

ytemp_tr_target = model_target.predict(xtr_target)
ytemp_ts_target = model_target.predict(xts_target)

xts_att = np.vstack((ytemp_tr_target,ytemp_ts_target))
yts_att = np.zeros((2*data_size,1))
yts_att[data_size:2*data_size] = 1
xts_att_truelabels = np.vstack((ytr_target,yts_target))
xts_att_dict = {'xts_att':xts_att,'yts_att':yts_att,'xts_att_truelabels':xts_att_truelabels}
fname = './att_test_data_'+str(data_size)
np.save(fname,xts_att_dict)
datafile = './data_adult_target_'+str(data_size)
np.save(datafile,target_rep)

/media/emanuel/sdd_sata/projetos/projeto_mestrado_ofc/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
W0000 00:00:1737033429.990473  828994 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/media/emanuel/sdd_sata/projetos/projeto_mestrado_ofc/.venv/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden (Dense)                  │ (None, 5)              │           530 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 536 (2.09 KB)

 Trainable params: 536 (2.09 KB)

 Non-trainable params: 0 (0.00 B)

None




For target model with training datasize = 10000
Training accuracy = 0.850800
Validation accuracy = 0.847700
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 303us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 265us/step


## Modelo Alvo Rastreado

In [16]:
import tensorflow.keras.optimizers as keras_optimizers

# Pré-processamento
x = x.astype('float32')
scaler = MinMaxScaler()
x = scaler.fit_transform(x)

# Hiperparâmetros
batch_size = 32
epochs = 100
lrate = 0.001
decay = 1e-7
data_size = 10000
ns = 5  # número de modelos sombra
nh = 5  # número de neurônios na camada oculta
nout = 1
seed = 7
np.random.seed(seed)

# Embaralhar índices
sh = np.arange(x.shape[0])
np.random.shuffle(sh)

# Divisão dos dados para o modelo alvo
xtr_target = x[sh[:data_size]]
ytr_target = y[sh[:data_size]]
xts_target = x[sh[data_size:data_size * 2]]
yts_target = y[sh[data_size:data_size * 2]]

shadow_rep = np.zeros((20,x.shape[0]-2*data_size))
sh1 = sh[2*data_size:]
xtr_att = np.zeros((2*data_size*ns,1))
ytr_att = np.zeros((2*data_size*ns,1))
xtr_att_truelabels = np.zeros((2*data_size*ns,))
# Resetar sessão Keras
k.clear_session()

# Definir modelo alvo
model_target = Sequential()
model_target.add(Dense(nh, input_shape=(x.shape[1],), activation='sigmoid', name='hidden'))
model_target.add(Dense(1, activation='sigmoid', name='output'))
opt = keras_optimizers.Adam(learning_rate=lrate, decay=decay)
model_target.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

# Treinamento do modelo alvo
print(model_target.summary())
hist_target = model_target.fit(
    xtr_target, ytr_target,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(xts_target, yts_target),
    shuffle=True,
    verbose=0
)

# Resultados do modelo alvo
print(f'\n\nFor target model with training datasize = {data_size}')
print(f'Training accuracy = {hist_target.history["accuracy"][-1]:.6f}')
print(f'Validation accuracy = {hist_target.history["val_accuracy"][-1]:.6f}')

# Salvar modelo alvo
model_target_name = f'UCI_Adult_target_{data_size}.h5'
model_target.save(model_target_name)

# Previsões do modelo alvo
ytemp_tr_target = model_target.predict(xtr_target)
ytemp_ts_target = model_target.predict(xts_target)

# Construir conjunto de ataque com colunas adicionais
xts_att = np.vstack((ytemp_tr_target, ytemp_ts_target))
yts_att = np.zeros((2 * data_size, 1))
yts_att[data_size:] = 1

ytr_target = ytr_target.reshape(-1, 1)  # (data_size, 1)
yts_target = yts_target.reshape(-1, 1) 
xts_att_truelabels = np.vstack((ytr_target, yts_target))

# Adicionar colunas para rastreamento
model_name_column = np.array(["target_model"] * len(xts_att)).reshape(-1, 1)
index_column = np.arange(len(xts_att)).reshape(-1, 1)

# Concatenar as informações ao conjunto de dados
xts_att_with_metadata = np.hstack((xts_att, yts_att, xts_att_truelabels, model_name_column, index_column))

# Salvar o conjunto de dados com as colunas adicionais
np.save(f'./att_test_data_{data_size}_with_metadata.npy', xts_att_with_metadata)

print(f"Attack data with metadata saved to './att_test_data_{data_size}_with_metadata.npy'")


/media/emanuel/sdd_sata/projetos/projeto_mestrado_ofc/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/media/emanuel/sdd_sata/projetos/projeto_mestrado_ofc/.venv/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden (Dense)                  │ (None, 5)              │           530 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 536 (2.09 KB)

 Trainable params: 536 (2.09 KB)

 Non-trainable params: 0 (0.00 B)

None




For target model with training datasize = 10000
Training accuracy = 0.850900
Validation accuracy = 0.847700
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 317us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 269us/step
Attack data with metadata saved to './att_test_data_10000_with_metadata.npy'


In [18]:
df_trace_data = pd.DataFrame(xts_att_with_metadata)

print(df_trace_data)

                  0    1  2             3      4
0         0.5227035  0.0  1  target_model      0
1         0.6208135  0.0  0  target_model      1
2         0.5483357  0.0  0  target_model      2
3       0.044427812  0.0  0  target_model      3
4         0.5909332  0.0  1  target_model      4
...             ...  ... ..           ...    ...
19995  0.0023097068  1.0  0  target_model  19995
19996  0.0045203026  1.0  0  target_model  19996
19997    0.02462684  1.0  0  target_model  19997
19998   0.062131066  1.0  0  target_model  19998
19999    0.55131245  1.0  1  target_model  19999

[20000 rows x 5 columns]


## Modelo Sombra Original

In [13]:
for i in np.arange(ns):
    
    np.random.shuffle(sh1)
    shadow_rep[i,:] = sh1
    xtr_shadow = x[sh1[:data_size]]
    ytr_shadow = y[sh1[:data_size]]
    xts_shadow = x[sh1[data_size:2*data_size]]
    yts_shadow = y[sh1[data_size:2*data_size]]
    
    model_shadow = Sequential()
    model_shadow.add(Dense(nh, input_shape =(x.shape[1],), activation='sigmoid', name = 'hidden'))
    model_shadow.add(Dense(1, activation='sigmoid', name = 'output'))
    opt = keras.optimizers.Adam(learning_rate=lrate, decay=decay)
    model_shadow.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    if i == 0:
        print("Shadow Model Summary")
        print(model_shadow.summary())
    hist_shadow = model_shadow.fit(xtr_shadow, ytr_shadow,
                  batch_size = batch_size,
                  epochs = epochs,
                  validation_data=(xts_shadow, yts_shadow), shuffle=True, verbose=0)
    print("Shadow model no: %d"%i)
    print('\n\nFor shadow model with training datasize = %d'%data_size)
    print('Training accuracy = %f'%hist_shadow.history['accuracy'][-1])
    print('Validation accuracy = %f'%hist_shadow.history['val_accuracy'][-1])
    
    ytemp11 = model_shadow.predict(xtr_shadow)
    ytemp22 = model_shadow.predict(xts_shadow)
    
    model_shadow_name = 'UCI_Adult_shadow_'+str(data_size)+'_'+str(i)+'.h5'
    print(model_shadow_name)
    model_shadow.save(model_shadow_name)
    
    xtr_att[i*2*data_size:(i+1)*2*data_size] = np.vstack((ytemp11,ytemp22))
    ytr_att[((i*2)+1)*data_size:(i+1)*2*data_size] = 1
    xtr_att_truelabels[i*2*data_size:(i+1)*2*data_size] = np.hstack((ytr_shadow,yts_shadow))
    
datafile = './data_adult_shadow_'+str(data_size)
np.save(datafile,shadow_rep)
xtr_att_dict = {'xtr_att':xtr_att,'ytr_att':ytr_att,'xtr_att_truelabels':xtr_att_truelabels}
fname = './att_train_data_'+str(data_size)
np.save(fname,xtr_att_dict)

Shadow Model Summary


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden (Dense)                  │ (None, 5)              │           530 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 536 (2.09 KB)

 Trainable params: 536 (2.09 KB)

...

UCI_Adult_shadow_10000_0.h5
Shadow model no: 1


For shadow model with training datasize = 10000
Training accuracy = 0.857800
Validation accuracy = 0.849600
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 319us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 285us/step


UCI_Adult_shadow_10000_1.h5
Shadow model no: 2


For shadow model with training datasize = 10000
Training accuracy = 0.859800
Validation accuracy = 0.848600
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 325us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 335us/step


UCI_Adult_shadow_10000_2.h5
Shadow model no: 3


For shadow model with training datasize = 10000
Training accuracy = 0.863400
Validation accuracy = 0.845900
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 323us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 284us/step


UCI_Adult_shadow_10000_3.h5
Shadow model no: 4


For shadow model with training datasize = 10000
Training accuracy = 0.857600
Validation accuracy = 0.849200
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 388us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 297us/step


UCI_Adult_shadow_10000_4.h5


## Modelo sombras rastreado

In [17]:
for i in np.arange(ns):
    np.random.shuffle(sh1)
    shadow_rep[i, :] = sh1
    
    xtr_shadow = x[sh1[:data_size]]
    ytr_shadow = y[sh1[:data_size]]
    xts_shadow = x[sh1[data_size:2 * data_size]]
    yts_shadow = y[sh1[data_size:2 * data_size]]
    
    # Criar e compilar o modelo sombra
    model_shadow = Sequential()
    model_shadow.add(Dense(nh, input_shape=(x.shape[1],), activation='sigmoid', name='hidden'))
    model_shadow.add(Dense(1, activation='sigmoid', name='output'))
    opt = keras.optimizers.Adam(learning_rate=lrate, decay=decay)
    model_shadow.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    if i == 0:
        print("Shadow Model Summary")
        print(model_shadow.summary())
    
    # Treinar o modelo sombra
    hist_shadow = model_shadow.fit(
        xtr_shadow, ytr_shadow,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(xts_shadow, yts_shadow),
        shuffle=True,
        verbose=0
    )
    print("Shadow model no: %d" % i)
    print('\n\nFor shadow model with training datasize = %d' % data_size)
    print('Training accuracy = %f' % hist_shadow.history['accuracy'][-1])
    print('Validation accuracy = %f' % hist_shadow.history['val_accuracy'][-1])
    
    # Previsões do modelo sombra
    ytemp11 = model_shadow.predict(xtr_shadow)
    ytemp22 = model_shadow.predict(xts_shadow)
    
    # Salvar o modelo sombra
    model_shadow_name = f'UCI_Adult_shadow_{data_size}_{i}.h5'
    print(model_shadow_name)
    model_shadow.save(model_shadow_name)
    
    # Concatenar previsões
    xtr_att[i * 2 * data_size:(i + 1) * 2 * data_size] = np.vstack((ytemp11, ytemp22))
    ytr_att[(i * 2 + 1) * data_size:(i + 1) * 2 * data_size] = 1
    xtr_att_truelabels[i * 2 * data_size:(i + 1) * 2 * data_size] = np.hstack((ytr_shadow, yts_shadow))
    
    # Adicionar colunas de metadados
    model_name_column = np.array([f'shadow_model_{i}'] * (2 * data_size)).reshape(-1, 1)
    index_column = np.arange(i * 2 * data_size, (i + 1) * 2 * data_size).reshape(-1, 1)
    
    # Concatenar metadados aos dados de ataque
    xtr_att_with_metadata = np.hstack((
        xtr_att[i * 2 * data_size:(i + 1) * 2 * data_size],
        ytr_att[i * 2 * data_size:(i + 1) * 2 * data_size].reshape(-1, 1),
        xtr_att_truelabels[i * 2 * data_size:(i + 1) * 2 * data_size].reshape(-1, 1),
        model_name_column,
        index_column
    ))
    
    # Salvar dados com metadados para cada modelo sombra
    fname = f'./att_train_data_shadow_{data_size}_model_{i}_with_metadata.npy'
    np.save(fname, xtr_att_with_metadata)
    print(f"Saved shadow model {i} data with metadata to {fname}")

# Salvar informações gerais dos modelos sombra
datafile = f'./data_adult_shadow_{data_size}'
np.save(datafile, shadow_rep)
print(f"Saved shadow models representation to {datafile}")


Shadow Model Summary


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden (Dense)                  │ (None, 5)              │           530 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 536 (2.09 KB)

 Trainable params: 536 (2.09 KB)

...

UCI_Adult_shadow_10000_0.h5
Saved shadow model 0 data with metadata to ./att_train_data_shadow_10000_model_0_with_metadata.npy
Shadow model no: 1


For shadow model with training datasize = 10000
Training accuracy = 0.858100
Validation accuracy = 0.851300
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 315us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 275us/step


UCI_Adult_shadow_10000_1.h5
Saved shadow model 1 data with metadata to ./att_train_data_shadow_10000_model_1_with_metadata.npy
Shadow model no: 2


For shadow model with training datasize = 10000
Training accuracy = 0.856900
Validation accuracy = 0.848600
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 310us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 278us/step


UCI_Adult_shadow_10000_2.h5
Saved shadow model 2 data with metadata to ./att_train_data_shadow_10000_model_2_with_metadata.npy
Shadow model no: 3


For shadow model with training datasize = 10000
Training accuracy = 0.862200
Validation accuracy = 0.845600
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 306us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 263us/step


UCI_Adult_shadow_10000_3.h5
Saved shadow model 3 data with metadata to ./att_train_data_shadow_10000_model_3_with_metadata.npy
Shadow model no: 4


For shadow model with training datasize = 10000
Training accuracy = 0.858200
Validation accuracy = 0.849100
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 302us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 276us/step


UCI_Adult_shadow_10000_4.h5
Saved shadow model 4 data with metadata to ./att_train_data_shadow_10000_model_4_with_metadata.npy
Saved shadow models representation to ./data_adult_shadow_10000


# Sombra Rastreado 2

In [ ]:
# Inicializando listas para armazenar os dados de todos os modelos sombra
all_xtr_att = []
all_ytr_att = []
all_xtr_att_truelabels = []
all_model_names = []
all_indices = []

for i in np.arange(ns):
    np.random.shuffle(sh1)
    shadow_rep[i, :] = sh1

    xtr_shadow = x[sh1[:data_size]]
    ytr_shadow = y[sh1[:data_size]]
    xts_shadow = x[sh1[data_size:2 * data_size]]
    yts_shadow = y[sh1[data_size:2 * data_size]]

    # Criar e compilar o modelo sombra
    model_shadow = Sequential()
    model_shadow.add(Dense(nh, input_shape=(x.shape[1],), activation='sigmoid', name='hidden'))
    model_shadow.add(Dense(1, activation='sigmoid', name='output'))
    opt = keras.optimizers.Adam(learning_rate=lrate, decay=decay)
    model_shadow.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    if i == 0:
        print("Shadow Model Summary")
        print(model_shadow.summary())

    # Treinar o modelo sombra
    hist_shadow = model_shadow.fit(
        xtr_shadow, ytr_shadow,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(xts_shadow, yts_shadow),
        shuffle=True,
        verbose=0
    )

    print("Shadow model no: %d" % i)
    print('\n\nFor shadow model with training datasize = %d' % data_size)
    print('Training accuracy = %f' % hist_shadow.history['accuracy'][-1])
    print('Validation accuracy = %f' % hist_shadow.history['val_accuracy'][-1])

    # Previsões do modelo sombra
    ytemp11 = model_shadow.predict(xtr_shadow)
    ytemp22 = model_shadow.predict(xts_shadow)

    # Salvar o modelo sombra
    model_shadow_name = f'UCI_Adult_shadow_{data_size}_{i}.h5'
    print(model_shadow_name)
    model_shadow.save(model_shadow_name)

    # Acumular previsões e metadados para os dados de ataque
    all_xtr_att.append(np.vstack((ytemp11, ytemp22)))
    all_ytr_att.append(np.ones(2 * data_size))  # Assumindo que todos os rótulos para modelos sombra são 1
    all_xtr_att_truelabels.append(np.hstack((ytr_shadow, yts_shadow)))
    all_model_names.append([f'shadow_model_{i}'] * (2 * data_size))  # Nome do modelo sombra
    all_indices.append(np.arange(i * 2 * data_size, (i + 1) * 2 * data_size))

# Agora, combine os dados acumulados
xtr_att_combined = np.vstack(all_xtr_att)
ytr_att_combined = np.hstack(all_ytr_att)
xtr_att_truelabels_combined = np.hstack(all_xtr_att_truelabels)
model_names_combined = np.hstack(all_model_names)
indices_combined = np.hstack(all_indices)

# Concatenando os metadados ao conjunto de dados final
xtr_att_with_metadata_combined = np.hstack((
    xtr_att_combined,
    ytr_att_combined.reshape(-1, 1),
    xtr_att_truelabels_combined.reshape(-1, 1),
    model_names_combined.reshape(-1, 1),
    indices_combined.reshape(-1, 1)
))

# Salvar o conjunto de dados combinado com metadados
fname = f'./att_train_data_combined_with_metadata_{data_size}.npy'
np.save(fname, xtr_att_with_metadata_combined)
print(f"Saved all shadow models data with metadata to {fname}")

# Salvar as representações dos modelos sombra (índices das amostras)
datafile = f'./data_adult_shadow_combined_{data_size}.npy'
np.save(datafile, shadow_rep)
print(f"Saved combined shadow models representation to {datafile}")


## Modelo de Ataque

In [14]:
model_attack = Sequential()
model_attack.add(Dense(nh, input_shape = (xtr_att.shape[1],), activation='sigmoid', name = 'hidden'))
model_attack.add(Dense(1, activation='sigmoid', name = 'output'))
opt = keras.optimizers.Adam(learning_rate=lrate, decay=decay)
model_attack.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
print("Attack Model Summary")
print(model_attack.summary())
hist_attack = model_attack.fit(xtr_att, ytr_att,
                  batch_size = batch_size,
                  epochs = epochs,
                  validation_data=(xts_att, yts_att), shuffle=True, verbose=0)
print('\n\nFor attack model with training datasize = %d'%xtr_att.shape[0])
print('Training accuracy = %f'%hist_attack.history['accuracy'][-1])
print('Validation accuracy = %f'%hist_attack.history['val_accuracy'][-1])
y_score = model_attack.predict(xts_att)
average_precision = average_precision_score(yts_att, y_score)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

Attack Model Summary


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden (Dense)                  │ (None, 5)              │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16 (64.00 B)

 Trainable params: 16 (64.00 B)

 Non-trainable params: 0 (0.00 B)

None


For attack model with training datasize = 100000
Training accuracy = 0.502230
Validation accuracy = 0.500000
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 257us/step
Average precision-recall score: 0.50


# Ataque com metadados

In [ ]:
# Carregar os dados combinados (com metadados) gerados pelo modelo sombra
xtr_att_with_metadata = np.load(f'./att_train_data_combined_with_metadata_{data_size}.npy')

# As previsões estão nas primeiras colunas de `xtr_att_with_metadata`, enquanto os metadados estão nas últimas colunas
xtr_att = xtr_att_with_metadata[:, :-4]  # Excluindo as 4 últimas colunas (metadados)
ytr_att = xtr_att_with_metadata[:, -4]   # A coluna dos rótulos de ataque está no 3º índice
xtr_att_truelabels = xtr_att_with_metadata[:, -3]  # As labels verdadeiras de xtr_att

# A estrutura de `xts_att` deve ser carregada da mesma forma
xts_att_with_metadata = np.load(f'./att_test_data_combined_with_metadata_{data_size}.npy')
xts_att = xts_att_with_metadata[:, :-4]  # Excluindo as 4 últimas colunas (metadados)
yts_att = xts_att_with_metadata[:, -4]   # A coluna dos rótulos de ataque está no 3º índice
xts_att_truelabels = xts_att_with_metadata[:, -3]  # As labels verdadeiras de xts_att

# Definir o modelo de ataque
model_attack = Sequential()
model_attack.add(Dense(nh, input_shape=(xtr_att.shape[1],), activation='sigmoid', name='hidden'))
model_attack.add(Dense(1, activation='sigmoid', name='output'))

# Configurar o otimizador e a função de perda
opt = keras.optimizers.Adam(learning_rate=lrate, decay=decay)
model_attack.compile(loss='binary_crossentropy',
                     optimizer=opt,
                     metrics=['accuracy'])

print("Attack Model Summary")
print(model_attack.summary())

# Treinar o modelo de ataque com os dados de treinamento
hist_attack = model_attack.fit(
    xtr_att, ytr_att,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(xts_att, yts_att),
    shuffle=True,
    verbose=0
)

# Exibir resultados do treinamento
print(f'\n\nFor attack model with training datasize = {xtr_att.shape[0]}')
print(f'Training accuracy = {hist_attack.history["accuracy"][-1]}')
print(f'Validation accuracy = {hist_attack.history["val_accuracy"][-1]}')

# Fazer previsões com o modelo de ataque
y_score = model_attack.predict(xts_att)

# Calcular o "average precision-recall score"
average_precision = average_precision_score(yts_att, y_score)
print(f'Average precision-recall score: {average_precision:0.2f}')


In [22]:
df_xtr_att = pd.DataFrame(xtr_att)
df_xtr_att

,0
0,0.710870
1,0.343707
2,0.001210
3,0.635679
4,0.763651
...,...
99995,0.009545
99996,0.356616
99997,0.727942
99998,0.746035


In [23]:
df_xts_att = pd.DataFrame(xts_att)
df_xts_att

,0
0,0.522704
1,0.620813
2,0.548336
3,0.044428
4,0.590933
...,...
19995,0.002310
19996,0.004520
19997,0.024627
19998,0.062131


In [ ]:
!pip install shap

In [ ]:
import shap

explainer = shap.Explainer(model_attack, xtr_att)

shap_values = explainer(xts_att[:1000])


In [ ]:
shap.plots.waterfall(shap_values[145])

In [ ]:
for i in range(3):
    shap.plots.waterfall(shap_values[i])

In [ ]:
shap.summary_plot(shap_values)